In [1]:
import json
import pathlib

In [2]:
cd /Users/davidnicholson/Documents/repos/coding/vocalpy/vocalpy

/Users/davidnicholson/Documents/repos/coding/vocalpy/vocalpy


In [3]:
ls tests/data-for-tests/source/audio_cbin_annot_notmat/gy6or6/

032312/ 032412/


In [4]:
import evfuncs
import vocalpy as voc

data_dir = 'tests/data-for-tests/source/audio_cbin_annot_notmat/gy6or6/032312/'

cbin_paths = voc.paths.from_dir(data_dir, 'cbin')
audios = [voc.Audio.read(cbin_path) for cbin_path in cbin_paths]

audios = [voc.Audio.read(cbin_path) for cbin_path in cbin_paths]


segment_params = {
    'threshold': 1500,
    'min_syl_dur': 0.01,
    'min_silent_dur': 0.006,
}
segmenter = voc.Segmenter(
    callback=evfuncs.segment_song, 
    segment_params=segment_params
)

seqs = segmenter.segment(audios)

seq_dataset = voc.dataset.SequenceDataset(sequences=seqs)

seq_dataset.to_sqlite(db_name='gy6or6-032312.db', replace=True)

[########################################] | 100% Completed | 106.05 ms


In [5]:
other_seq_dataset = voc.dataset.SequenceDataset.from_sqlite(db_name='gy6or6-032312.db')

In [6]:
other_seq_dataset == seq_dataset

True

In [7]:
other_seq_dataset

SequenceDataset(sequences=[Sequence(units=[Unit(onset=2.18934375, offset=2.21, label='-', audio=None, spectrogram=None), Unit(onset=2.346125, offset=2.373125, label='-', audio=None, spectrogram=None), Unit(onset=2.50471875, offset=2.51546875, label='-', audio=None, spectrogram=None), Unit(onset=2.81909375, offset=2.84740625, label='-', audio=None, spectrogram=None), Unit(onset=3.44878125, offset=3.4676875, label='-', audio=None, spectrogram=None), Unit(onset=3.51753125, offset=3.565875, label='-', audio=None, spectrogram=None), Unit(onset=3.6085625, offset=3.6390625, label='-', audio=None, spectrogram=None), Unit(onset=3.68625, offset=3.7098125, label='-', audio=None, spectrogram=None), Unit(onset=3.747875, offset=3.761125, label='-', audio=None, spectrogram=None), Unit(onset=4.0930625, offset=4.123375, label='-', audio=None, spectrogram=None), Unit(onset=4.75184375, offset=4.7740625, label='-', audio=None, spectrogram=None), Unit(onset=4.8950625, offset=4.94246875, label='-', audio=No

In [9]:
with Session(engine) as session, session.begin():
    seg_params_stmt = (
        select(voc.dataset.models.sequence.SegmentParams)
            .order_by(voc.dataset.models.sequence.SegmentParams.id)
    )
    seg_params_result = session.scalars(seg_params_stmt).all()

    segment_params = []
    for model_segment_params in seg_params_result:
        path = model_segment_params.path
        with pathlib.Path(path).open('r') as fp:
            this_seg_params = json.load(fp)
        segment_params.append(this_seg_params)

    audios = []
    audio_stmt = (
        select(voc.dataset.models.sequence.Audio)
            .order_by(voc.dataset.models.sequence.Audio.id)
    )
    audio_result = session.scalars(audio_stmt).all()
    for model_audio in audio_result:
        audios.append(
            voc.Audio(path=model_audio.path)
        )
    
    seqs_stmt = (
        select(voc.dataset.models.sequence.Sequence)
            .order_by(voc.dataset.models.sequence.Sequence.id)
    )
    seqs_result = session.scalars(seqs_stmt).all()

    seqs = []
    for model_sequence in seqs_result:
        units_stmt = (
            select(voc.dataset.models.sequence.Unit)
                .where(voc.dataset.models.sequence.Unit.sequence_id == model_sequence.id)
                .order_by(voc.dataset.models.sequence.Unit.id)
        )
        seq_units_result = session.scalars(units_stmt).all()

        units = []
        for model_unit in seq_units_result:
            unit = voc.Unit(onset=model_unit.onset, offset=model_unit.offset, label=model_unit.label)
            units.append(unit)

        seq = voc.Sequence(
            units=units,
            audio=audios[model_sequence.audio_id - 1],
            method=model_sequence.method,
            segment_params=segment_params[model_sequence.segment_params_id - 1]
        )
        seqs.append(seq)

2023-05-11 09:43:33,638 INFO sqlalchemy.engine.Engine BEGIN (implicit)
2023-05-11 09:43:33,652 INFO sqlalchemy.engine.Engine SELECT segment_params.id, segment_params.path 
FROM segment_params ORDER BY segment_params.id
2023-05-11 09:43:33,655 INFO sqlalchemy.engine.Engine [cached since 8.124e+04s ago] ()
2023-05-11 09:43:33,672 INFO sqlalchemy.engine.Engine SELECT audios.id, audios.path 
FROM audios ORDER BY audios.id
2023-05-11 09:43:33,673 INFO sqlalchemy.engine.Engine [cached since 8.124e+04s ago] ()
2023-05-11 09:43:33,677 INFO sqlalchemy.engine.Engine SELECT sequences.id, sequences.audio_id, sequences.onset, sequences."offset", sequences.method, sequences.segment_params_id 
FROM sequences ORDER BY sequences.id
2023-05-11 09:43:33,679 INFO sqlalchemy.engine.Engine [cached since 8.124e+04s ago] ()
2023-05-11 09:43:33,687 INFO sqlalchemy.engine.Engine SELECT units.id, units.sequence_id, units.onset, units."offset", units.label 
FROM units 
WHERE units.sequence_id = ? ORDER BY units.i

In [10]:
seqs

[Sequence(units=[Unit(onset=2.18934375, offset=2.21, label='-', audio=None, spectrogram=None), Unit(onset=2.346125, offset=2.373125, label='-', audio=None, spectrogram=None), Unit(onset=2.50471875, offset=2.51546875, label='-', audio=None, spectrogram=None), Unit(onset=2.81909375, offset=2.84740625, label='-', audio=None, spectrogram=None), Unit(onset=3.44878125, offset=3.4676875, label='-', audio=None, spectrogram=None), Unit(onset=3.51753125, offset=3.565875, label='-', audio=None, spectrogram=None), Unit(onset=3.6085625, offset=3.6390625, label='-', audio=None, spectrogram=None), Unit(onset=3.68625, offset=3.7098125, label='-', audio=None, spectrogram=None), Unit(onset=3.747875, offset=3.761125, label='-', audio=None, spectrogram=None), Unit(onset=4.0930625, offset=4.123375, label='-', audio=None, spectrogram=None), Unit(onset=4.75184375, offset=4.7740625, label='-', audio=None, spectrogram=None), Unit(onset=4.8950625, offset=4.94246875, label='-', audio=None, spectrogram=None), Uni